In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

2025-02-28 15:12:38.975973: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 15:12:39.470080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740773559.818435    1504 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740773559.919814    1504 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 15:12:40.487160: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available: 1


In [2]:
print(tf.config.experimental.list_logical_devices('GPU'))


[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


I0000 00:00:1740773625.326226    1504 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2256 MB memory:  -> device: 0, name: NVIDIA T500, pci bus id: 0000:01:00.0, compute capability: 7.5


In [4]:
# Enable logging of device placement
tf.debugging.set_log_device_placement(True)

# Create and multiply tensors
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0]])
    b = tf.constant([[4.0, 5.0, 6.0]])
    c = tf.matmul(a, b, transpose_b=True)

print(c)


tf.Tensor([[32.]], shape=(1, 1), dtype=float32)


In [5]:
print("TF will use GPU?" , tf.test.is_built_with_cuda())  # Should return True
print("GPU in use:", tf.config.list_physical_devices('GPU'))  # Should list your GPU


TF will use GPU? True
GPU in use: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [48]:
def rail_fence_decrypt(ciphertext, rails, offset=0):
    length = len(ciphertext)
    cycle = 2 * (rails - 1)  # Full zigzag cycle length

    # Step 1: Create an empty rail matrix
    rail = [[' ' for _ in range(length)] for _ in range(rails)]

    # Step 2: Mark the zigzag pattern
    row, col = offset, 0
    down = True  # Direction flag

    for i in range(length):
        rail[row][col] = '*'  # Mark where letters should go
        col += 1

        if down:
            row += 1
        else:
            row -= 1

        if row == 0:
            down = True
        elif row == rails - 1:
            down = False

    # Step 3: Fill the rails with the ciphertext
    index = 0
    for r in range(rails):
        for c in range(length):
            if rail[r][c] == '*' and index < length:
                rail[r][c] = ciphertext[index]
                index += 1

    # Step 4: Read the rails in zigzag order
    result = []
    row, col = offset, 0
    down = True

    for i in range(length):
        result.append(rail[row][col])
        col += 1

        if down:
            row += 1
        else:
            row -= 1

        if row == 0:
            down = True
        elif row == rails - 1:
            down = False

    # Step 5: Print the rail pattern
    print("\nRail Fence Pattern:")
    for r in rail:
        print("".join(r))

    return "".join(result)  # Remove spaces from empty slots

# Example usage
originaltext = "Rropnleimi gooFort oedalr eg efxrdn tgb sweeeMiW”etnhst iiiw erett hm“n hw br ftevdalter,t gwooohilh"
ciphertext = "Rropnleimi gooFort oedalr eg efxrdn tgb sweeeMiW”etnhst iiiw erett hm“n hw br ftevdalter,t gwooohilh"
rails = 5
offset = 0  # Try adjusting this

decrypted_text = rail_fence_decrypt(ciphertext, rails, offset)
print("\nDecrypted Text:", decrypted_text)



Rail Fence Pattern:
R       r       o       p       n       l       e       i       m       i               g       o   
 o     F o     r t       o     e d     a l     r       e g       e     f x     r d     n       t g  
  b       s   w   e   e   e   M   i   W   ”   e   t   n   h   s   t       i   i   i   w       e   r 
   e t     t       h m     “ n       h     w       b r       f     t e     v d     a l     t e     r
    ,       t               g       w       o       o       o       h       i       l       h       

Decrypted Text: Robe,t Frostt wroteh me poe“gnMendi whWall”wo ere tborneigh ofs metthe fixividir diallwng  theetogrr
